In [5]:
import time
from utils import *

# from data_preprocessing.ihdp import * 
# from data_preprocessing.lalonde import *
# from data_preprocessing.lbidd import *
# from data_preprocessing.synthetic import *
# from data_preprocessing.twins import *

k = 2
# ci_estimators = ['sl', 'tl', 'xl', 'dml', 'orf', 'dr', 'sparse_dml', 'kernel_dml', 'CausalForestDML']


def causal_inference_analysis(model_y, model_t, str_causal_model,x, y, t, true_ate, true_ate_std, true_ite):
    causal_model = get_estimators(str_causal_model, model_y, model_t)

    start_time = time.time()
    causal_model.fit(y, t, X=x, W=None)
    run_time = time.time() - start_time

    estimated_ate = causal_model.ate(x)
    
    estimated_ite_values = causal_model.effect(x)

    tao_risk, mu_risk = calculate_risks(true_ate, estimated_ate, true_ite, estimated_ite_values)

    return {'est_ate': estimated_ate, 'mu_risk': mu_risk, 'tao_risk': tao_risk, 
            'true_ate_std': true_ate_std, 'run_time': run_time, 
            'model_t': model_t.__class__.__name__, 'model_y': model_y.__class__.__name__}, estimated_ite_values

    

NameError: name 'dml' is not defined

In [6]:
# classifiers = [GradientBoostingClassifier(),
#                RandomForestClassifier(),
#                LogisticRegression(),
#                LogisticRegressionCV(),
#                MLPClassifier(),
#                DecisionTreeClassifier()]

classifiers = [LogisticRegression()]

# regressors = [GradientBoostingRegressor(),
#             RandomForestRegressor(),
#             LinearRegression(),
#             ElasticNet(),
#             ElasticNetCV(),
#             Lasso(),
#             LassoLars(),
#             OrthogonalMatchingPursuit(),
#             Ridge(),
#             MLPRegressor(),
#             DecisionTreeRegressor()]

regressors = [LinearRegression()]

ci_estimators = ['dml']

data_dict = {'ihdp':load_ihdp()}
all_results = []
for key in data_dict:
    data, X, T, Y, true_ite, true_ATE, true_ATE_stderr, is_discrete = data_dict[key]
    my_list = regressors
    mt_list = classifiers if is_discrete else regressors
    for str_causal_model in ci_estimators:
        is_meta = False
        if str_causal_model in ['sl', 'xl', 'tl']:
                is_meta = True
        for model_y  in my_list:
            count = 0
            for model_t in mt_list:
                try:
                    temp_results = causal_inference_analysis(model_y, model_t, str_causal_model, X, Y, T, true_ATE, true_ATE_stderr, true_ite)
                    temp_results['data'] = key
                    all_results.append(temp_results)
                    results_df = pd.DataFrame(all_results)
                    results_df.to_csv(f'results/{data}_no_params_baselines.csv')
                    print(f"Completed running model_y: {model_y}, model_t: {model_t}, str_causal_model: {str_causal_model}")
                except Exception as e:
                    print(f"Error occurred while running {model_y}-{model_t} estimator with {str_causal_model} method: {str(e)}")
    





Error occurred while running LinearRegression()-LogisticRegression() estimator with dml method: 'tuple' object does not support item assignment


In [5]:
T

0      1
1      0
2      0
3      0
4      0
      ..
742    0
743    1
744    0
745    0
746    0
Name: treatment, Length: 747, dtype: int64

In [12]:
dml = LinearDML(model_t=model_t, model_y=model_y)
dml.fit(Y, T.astype(int), X=X, W=None)

In [13]:
dml.__class__.__name__


'LinearDML'

In [11]:
dml.ate

AssertionError: X was not None when fitting, so can't be none for score or effect

In [20]:
# dml = get_estimators(str_causal_model, model_y, model_t)
dml = LinearDML(model_t=DecisionTreeClassifier(), model_y=ElasticNet())
dml.fit(Y, T.astype(int), X=X, W=None)

In [12]:
T

0       True
1      False
2      False
3      False
4      False
       ...  
742    False
743     True
744    False
745    False
746    False
Name: treatment, Length: 747, dtype: bool

In [9]:
from econml.orf import DMLOrthoForest

In [7]:
import pandas as pd
import numpy as np

#The covariates data has 46 features
x = pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/TWINS/twin_pairs_X_3years_samesex.csv")

#The outcome data contains mortality of the lighter and heavier twin
y = pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/TWINS/twin_pairs_Y_3years_samesex.csv")

#The treatment data contains weight in grams of both the twins
t = pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/TWINS/twin_pairs_T_3years_samesex.csv")

lighter_columns = ['pldel', 'birattnd', 'brstate', 'stoccfipb', 'mager8',
       'ormoth', 'mrace', 'meduc6', 'dmar', 'mplbir', 'mpre5', 'adequacy',
       'orfath', 'frace', 'birmon', 'gestat10', 'csex', 'anemia', 'cardiac',
       'lung', 'diabetes', 'herpes', 'hydra', 'hemo', 'chyper', 'phyper',
       'eclamp', 'incervix', 'pre4000', 'preterm', 'renal', 'rh', 'uterine',
       'othermr', 'tobacco', 'alcohol', 'cigar6', 'drink5', 'crace',
       'data_year', 'nprevistq', 'dfageq', 'feduc6', 'infant_id_0',
       'dlivord_min', 'dtotord_min', 'bord_0',
       'brstate_reg', 'stoccfipb_reg', 'mplbir_reg']
heavier_columns = [ 'pldel', 'birattnd', 'brstate', 'stoccfipb', 'mager8',
       'ormoth', 'mrace', 'meduc6', 'dmar', 'mplbir', 'mpre5', 'adequacy',
       'orfath', 'frace', 'birmon', 'gestat10', 'csex', 'anemia', 'cardiac',
       'lung', 'diabetes', 'herpes', 'hydra', 'hemo', 'chyper', 'phyper',
       'eclamp', 'incervix', 'pre4000', 'preterm', 'renal', 'rh', 'uterine',
       'othermr', 'tobacco', 'alcohol', 'cigar6', 'drink5', 'crace',
       'data_year', 'nprevistq', 'dfageq', 'feduc6',
       'infant_id_1', 'dlivord_min', 'dtotord_min', 'bord_1',
       'brstate_reg', 'stoccfipb_reg', 'mplbir_reg']

#Since data has pair property,processing the data to get separate row for each twin so that each child can be treated as an instance
data = []

for i in range(len(t.values)):
    
    #select only if both <=2kg
    if t.iloc[i].values[1]>=2000 or t.iloc[i].values[2]>=2000:
        continue
    
    this_instance_lighter = list(x.iloc[i][lighter_columns].values)
    this_instance_heavier = list(x.iloc[i][heavier_columns].values)
    
    #adding weight
    this_instance_lighter.append(t.iloc[i].values[1])
    this_instance_heavier.append(t.iloc[i].values[2])
    
    #adding treatment, is_heavier
    this_instance_lighter.append(0)
    this_instance_heavier.append(1)
    
    #adding the outcome
    this_instance_lighter.append(y.iloc[i].values[1])
    this_instance_heavier.append(y.iloc[i].values[2])
    data.append(this_instance_lighter)
    data.append(this_instance_heavier)
cols = [ 'pldel', 'birattnd', 'brstate', 'stoccfipb', 'mager8',
       'ormoth', 'mrace', 'meduc6', 'dmar', 'mplbir', 'mpre5', 'adequacy',
       'orfath', 'frace', 'birmon', 'gestat10', 'csex', 'anemia', 'cardiac',
       'lung', 'diabetes', 'herpes', 'hydra', 'hemo', 'chyper', 'phyper',
       'eclamp', 'incervix', 'pre4000', 'preterm', 'renal', 'rh', 'uterine',
       'othermr', 'tobacco', 'alcohol', 'cigar6', 'drink5', 'crace',
       'data_year', 'nprevistq', 'dfageq', 'feduc6',
       'infant_id', 'dlivord_min', 'dtotord_min', 'bord',
       'brstate_reg', 'stoccfipb_reg', 'mplbir_reg','wt','treatment','outcome']
df = pd.DataFrame(columns=cols,data=data)
df = df.astype({"treatment":'bool'}, copy=False) #explicitly assigning treatment column as boolean 

df.fillna(value=df.mean(),inplace=True)    #filling the missing values
df.fillna(value=df.mode().loc[0],inplace=True)

data_1 = df[df["treatment"]==1]
data_0 = df[df["treatment"]==0]

ITE = data_1["outcome"] - data_0["outcome"]
ATE =  np.mean(ITE)


In [9]:
df.treatment.value_counts()

False    11984
True     11984
Name: treatment, dtype: int64

In [12]:
data = np.load('../datasets/IHDP/ihdp_splits_10iters_10folds.npz', allow_pickle=True)
# n_iters = options.iters if options.iters > 0 else splits.shape[0]


In [23]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.linear_model import (Lasso, LassoCV, ElasticNetCV, LogisticRegression,
                                  LogisticRegressionCV,LinearRegression,
                                  MultiTaskElasticNet,MultiTaskElasticNetCV)
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor,RandomForestRegressor,RandomForestClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

models_and_parameters = [    ('elastic', ElasticNetCV(), {}),    
('forest', RandomForestRegressor(),        {'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20],
        'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 1, 2, 3, 4, 5, 6, 7, 8, 9]}),
    ('gbf', GradientBoostingRegressor(),
        {"n_estimators": [100, 300, 500, 700, 1000],
        "max_depth": [2, 4, 6, 8, 10],
        "min_samples_split": [5, 10, 15, 20],
        "learning_rate": [0.001, 0.01, 0.1, 0.2, 1]}),
    ('nnet', MLPRegressor(),
        {'hidden_layer_sizes': [4, 8, 16, 32, 64, 128],
        'learning_rate_init': [0.0001, 0.001],
        'batch_size': [32, 64, 128, 250]})
]
models_dict = {name: (model, params) for name, model, params in models_and_parameters}

In [25]:
models_dict = {name: {'model': model, 'parameters': params} for name, model, params in models_and_parameters}

In [26]:
models_dict

{'elastic': {'model': ElasticNetCV(), 'parameters': {}},
 'forest': {'model': RandomForestRegressor(),
  'parameters': {'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20],
   'min_samples_leaf': [0.01,
    0.02,
    0.03,
    0.04,
    0.05,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9]}},
 'gbf': {'model': GradientBoostingRegressor(),
  'parameters': {'n_estimators': [100, 300, 500, 700, 1000],
   'max_depth': [2, 4, 6, 8, 10],
   'min_samples_split': [5, 10, 15, 20],
   'learning_rate': [0.001, 0.01, 0.1, 0.2, 1]}},
 'nnet': {'model': MLPRegressor(),
  'parameters': {'hidden_layer_sizes': [4, 8, 16, 32, 64, 128],
   'learning_rate_init': [0.0001, 0.001],
   'batch_size': [32, 64, 128, 250]}}}